# temporal ranking corr and fixing moving invariants

In [ ]:
import numpy as np
import torch
import sys
import os

import argparse
import numpy as np
import json

from umap.umap_ import find_ab_params
sys.path.append("..")
from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.data import NormalDataProvider
from singleVis.eval.evaluator import Evaluator
from singleVis.projector import TimeVisProjector
from singleVis.SingleVisualizationModel import VisModel
from singleVis.utils import *

In [ ]:
VIS_METHOD = "TimeVis" # DeepVisualInsight
CONTENT_PATH = "/home/xianglin/projects/DVI_data/resnet18_fmnist"
GPU_ID = "3"

In [ ]:
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

In [ ]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = VISUALIZATION_PARAMETER["VIS_MODEL_NAME"]
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

# Define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

In [ ]:
import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))

In [ ]:
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, classes=CLASSES, epoch_name="Epoch", verbose=1)
if PREPROCESS:
    data_provider._meta_data()
    if B_N_EPOCHS >0:
        data_provider._estimate_boundary(LEN//10, l_bound=L_BOUND)


model = VisModel(ENCODER_DIMS, DECODER_DIMS)
projector = TimeVisProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, device=DEVICE)

In [ ]:
evaluator = Evaluator(data_provider, projector)

In [ ]:
for s,e in [(1,25),(25,50),(1,50),(1,5),(23,27),(46,50)]:
    evaluator.eval_moving_invariants_train(s,e)
    evaluator.eval_moving_invariants_test(s,e)


In [ ]:
evaluator.eval_proj_invariants_train(180)

# temporal nn spearman ranking test

In [ ]:
def ranking_dist(a,b):  
    n = len(a)
    assert len(b) == n
    i, j = np.meshgrid(np.arange(n), np.arange(n))
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered / (n * (n - 1))

In [ ]:
def ranking_change(all_train_repr, low_repr, e_s, e_e, l):
    correct = 0
    pool = 0
    for idx in range(l):
        high_embeddings = all_train_repr[:,idx,:]
        low_embeddings = low_repr[:,idx,:]
        high_rank_s = np.argsort(np.linalg.norm(high_embeddings - high_embeddings[e_s-1], axis=1))
        high_rank_e = np.argsort(np.linalg.norm(high_embeddings - high_embeddings[e_e-1], axis=1))
        dist = ranking_dist(high_rank_s, high_rank_e)
        if dist >0.5:
            pool += 1
            low_rank_s = np.argsort(np.linalg.norm(low_embeddings - low_embeddings[e_s-1], axis=1))
            low_rank_e = np.argsort(np.linalg.norm(low_embeddings - low_embeddings[e_e-1], axis=1))
            low_dist = ranking_dist(low_rank_s, low_rank_e)
            if low_dist>0.5:
                correct+=1
    print(f'Radical Change in Low/High:\t{correct}/{pool}')


In [ ]:
EPOCH = 200
LEN = 50000

In [ ]:
all_train_repr = np.zeros((EPOCH,LEN,512))
for i in range(1,201,1):
    all_train_repr[i-1] = data_provider.train_representation(i)

In [ ]:
low_repr = np.zeros((EPOCH,LEN,2))
for e in range(EPOCH):
    low_repr[e] = projector.batch_project(e+1, all_train_repr[e])

In [ ]:
ranking_change(all_train_repr, low_repr, 100,150, LEN)

In [ ]:
from scipy import stats

In [ ]:
# shape (200, 50000, 512)
epochs = [i for i in range(EPOCH)]
corrs = np.zeros((EPOCH,500))
ps = np.zeros((EPOCH,500))
for i in range(500):
    high_embeddings = all_train_repr[:,i,:].squeeze()
    low_embeddings = low_repr[:,i,:].squeeze()

    for e in epochs:
        high_dists = np.linalg.norm(high_embeddings - high_embeddings[e], axis=1)
        low_dists = np.linalg.norm(low_embeddings - low_embeddings[e], axis=1)
        corr, p = stats.spearmanr(high_dists, low_dists)
        corrs[e][i] = corr
        ps[e][i] = p


In [ ]:
import matplotlib.pyplot as plt
mean_corr = np.mean(corrs, axis=1)
var_corr = np.var(corrs, axis=1)

mean_p = np.mean(ps, axis=1)
var_p = np.var(ps, axis=1)


p1 = plt.plot(epochs, mean_corr, "b.-", epochs, mean_p, "r+-")
p2 = plt.fill_between(epochs, mean_corr-var_corr, mean_corr+var_corr)
p3 = plt.fill_between(epochs, mean_p-var_p, mean_p+var_p)
plt.show

In [ ]:
import matplotlib.pyplot as plt
mean_corr = np.mean(corrs, axis=1)
var_corr = np.var(corrs, axis=1)

mean_p = np.mean(ps, axis=1)
var_p = np.var(ps, axis=1)


p1 = plt.plot(epochs, mean_corr, "b.-", epochs, mean_p, "r+-")
p2 = plt.fill_between(epochs, mean_corr-var_corr, mean_corr+var_corr)
p3 = plt.fill_between(epochs, mean_p-var_p, mean_p+var_p)
plt.show

In [ ]:
import matplotlib.pyplot as plt
mean_corr = np.mean(corrs, axis=1)
var_corr = np.var(corrs, axis=1)

mean_p = np.mean(ps, axis=1)
var_p = np.var(ps, axis=1)


p1 = plt.plot(epochs, mean_corr, "b-", epochs, mean_p, "r-")
p2 = plt.fill_between(epochs, mean_corr-var_corr, mean_corr+var_corr)
p3 = plt.fill_between(epochs, mean_p-var_p, mean_p+var_p)
plt.show

# fixing invarient and Moving invarient

In [ ]:
from scipy.special import softmax
from scipy import stats
from scipy.spatial.distance import cosine

In [ ]:
e_s = 2
e_t = 40

In [ ]:
train_data_s = data_provider.train_representation(e_s)
train_data_t = data_provider.train_representation(e_t)
pred_s = data_provider.get_pred(e_s, train_data_s)
pred_t = data_provider.get_pred(e_t, train_data_t)

labels = data_provider.train_labels(20)

model = trainer.model

low_s = model.encoder(torch.from_numpy(train_data_s).to(device=DEVICE).float()).detach().cpu().numpy()
low_t = model.encoder(torch.from_numpy(train_data_t).to(device=DEVICE).float()).detach().cpu().numpy()


## correlation between (kl div/js div/loss) and dists

In [ ]:
def kl_div(p, q):
    return stats.entropy(p, q, base=2)


def js_div(p, q):
    M = (p+q)/2
    return .5*kl_div(p, M)+.5*kl_div(q, M)

In [ ]:
# kl
softmax_s = softmax(pred_s, axis=1)
softmax_t = softmax(pred_t, axis=1)
kl_lists = [kl_div(softmax_s[i], softmax_t[i]) for i in range(len(softmax_t))]
dists = [cosine(low_t[i], low_s[i]) for i in range(len(low_s))]
corr, p = stats.spearmanr(kl_lists, dists)
corr, p

In [ ]:
# js
js_lists = [js_div(softmax_s[i], softmax_t[i]) for i in range(len(softmax_t))]
dists = [cosine(low_t[i], low_s[i]) for i in range(len(low_s))]
corr, p = stats.spearmanr(js_lists, dists)
corr, p

In [ ]:
# loss
loss_s = softmax_s[range(LEN), labels]
loss_t = softmax_t[range(LEN), labels]
loss_diff = np.abs(loss_s-loss_t)

dists = [cosine(low_t[i], low_s[i]) for i in range(len(low_s))]
corr, p = stats.spearmanr(js_lists, dists)
corr, p

## fixing and moving comparison

In [ ]:
# normalize low dimension distance
mean_x = np.mean(low_t[:, 0])
mean_y = np.mean(low_t[:, 1])
low_t = low_t - [mean_x, mean_y]
low_s = low_s - [mean_x, mean_y]

max_n = np.linalg.norm(low_t)
low_t = low_t/max_n*100
low_s = low_s/max_n*100


In [ ]:
np.linalg.norm((train_data_s-train_data_t),axis=1).mean(), np.linalg.norm((low_s-low_t),axis=1).mean()

In [ ]:
selected = np.argsort(kl_lists)
print("kl div")
print("fixing\t", np.linalg.norm((train_data_s-train_data_t)[selected[:100]],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected[:100]],axis=1).mean())
print("moving\t", np.linalg.norm((train_data_s-train_data_t)[selected[-100:]],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected[-100:]],axis=1).mean())

In [ ]:
selected = np.argsort(js_lists)
print("js div")
print("fixing\t", np.linalg.norm((train_data_s-train_data_t)[selected[:100]],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected[:100]],axis=1).mean())
print("moving\t", np.linalg.norm((train_data_s-train_data_t)[selected[-100:]],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected[-100:]],axis=1).mean())

In [ ]:
selected = np.argsort(loss_diff)
print("loss diff")
print("fixing\t", np.linalg.norm((train_data_s-train_data_t)[selected[:100]],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected[:100]],axis=1).mean())
print("moving\t", np.linalg.norm((train_data_s-train_data_t)[selected[-100:]],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected[-100:]],axis=1).mean())

## inside(outside)-class moving dists

In [ ]:
p_s = pred_s.argmax(axis=1)
p_t = pred_t.argmax(axis=1)
for i in range(10):
    selected = np.logical_and(p_s==i, p_s==p_t)
    print(i, np.linalg.norm((train_data_s-train_data_t)[selected],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected],axis=1).mean())

In [ ]:
for i in range(10):
    selected = np.logical_and(p_s==i, p_s!=p_t)
    print(i, np.linalg.norm((train_data_s-train_data_t)[selected],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected],axis=1).mean())

## npr and prediction based measurements

In [ ]:
from singleVis.backend import find_neighbor_preserving_rate

npr = find_neighbor_preserving_rate(train_data_s, train_data_t, 15)
selected_fix = np.logical_and(npr>0.1, p_s==p_t)
for i in range(10):
    selected = np.logical_and(selected_fix, p_s==i)
    print(i, np.linalg.norm((train_data_s-train_data_t)[selected],axis=1).mean(), np.linalg.norm((low_s-low_t)[selected],axis=1).mean())

# fixing and moving invariants

## moving

In [ ]:
e_s = 1
e_t = 3

In [ ]:
train_data_s = data_provider.train_representation(e_s)
train_data_t = data_provider.train_representation(e_t)
pred_s = data_provider.get_pred(e_s, train_data_s)
pred_t = data_provider.get_pred(e_t, train_data_t)

labels = data_provider.train_labels(20)

model = trainer.model

low_s = model.encoder(torch.from_numpy(train_data_s).to(device=DEVICE).float()).detach().cpu().numpy()
low_t = model.encoder(torch.from_numpy(train_data_t).to(device=DEVICE).float()).detach().cpu().numpy()

In [ ]:
from singleVis.utils import is_B
s_B = is_B(pred_s)
t_B = is_B(pred_t)

predictions_s = pred_s.argmax(1)
predictions_t = pred_t.argmax(1)

confident_sample = np.logical_and(np.logical_not(s_B),np.logical_not(t_B))
diff_pred = predictions_s!=predictions_t

selected = np.logical_and(diff_pred, confident_sample)


In [ ]:
from singleVis.visualizer import visualizer
resolution = 500
vis = visualizer(data_provider, trainer.model, resolution, 10, classes, cmap='tab10')

In [ ]:
grid_view_s, decision_view_s = vis.get_epoch_decision_view(e_s, resolution)
grid_view_t, decision_view_t = vis.get_epoch_decision_view(e_t, resolution)

grid_view_s = grid_view_s.reshape(resolution*resolution, -1)
grid_view_t = grid_view_t.reshape(resolution*resolution, -1)

grid_samples_s = trainer.model.decoder(grid_view_s).cpu().detach().numpy()
grid_samples_t = trainer.model.decoder(grid_view_t).cpu().detach().numpy()

grid_pred_s = data_provider.get_pred(e_s, grid_samples_s)+1e-8
grid_pred_t = data_provider.get_pred(e_t, grid_samples_t)+1e-8


In [ ]:
grid_view_s = grid_view_s.cpu().detach().numpy()
grid_view_t = grid_view_t.cpu().detach().numpy()

In [ ]:
grid_s_B = is_B(grid_pred_s)
grid_t_B = is_B(grid_pred_t)

grid_predictions_s = grid_pred_s.argmax(1)
grid_predictions_t = grid_pred_t.argmax(1)


In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
high_neigh = NearestNeighbors(n_neighbors=1, radius=0.4)
high_neigh.fit(grid_view_s)
knn_dists, knn_indices = high_neigh.kneighbors(low_s, n_neighbors=1, return_distance=True)

close_s_pred = grid_predictions_s[knn_indices].squeeze()
close_s_B = grid_s_B[knn_indices].squeeze()

In [ ]:
s_true = np.logical_and(close_s_pred==predictions_s, close_s_B == s_B)
np.sum(s_true[selected]), np.sum(selected)

In [ ]:
high_neigh = NearestNeighbors(n_neighbors=1, radius=0.4)
high_neigh.fit(grid_view_t)
knn_dists, knn_indices = high_neigh.kneighbors(low_t, n_neighbors=1, return_distance=True)

close_t_pred = grid_predictions_t[knn_indices].squeeze()
close_t_B = grid_t_B[knn_indices].squeeze()
np.sum(grid_t_B), np.sum(t_B)

In [ ]:
t_true = np.logical_and(close_t_pred==predictions_t, close_t_B == t_B)
np.sum(t_true[selected]), np.sum(selected)

In [ ]:
np.sum(np.logical_and(s_true[selected], t_true[selected])), np.sum(selected)

## fixing

In [ ]:
# fixing invariants
from scipy.special import softmax
from scipy import stats
from scipy.spatial.distance import cosine

In [ ]:
def kl_div(p, q):
    return stats.entropy(p, q, base=2)


def js_div(p, q):
    M = (p+q)/2
    return .5*kl_div(p, M)+.5*kl_div(q, M)

In [ ]:
e_s = 1
e_t = 3

In [ ]:
train_data_s = data_provider.train_representation(e_s)
train_data_t = data_provider.train_representation(e_t)
pred_s = data_provider.get_pred(e_s, train_data_s)
pred_t = data_provider.get_pred(e_t, train_data_t)
softmax_s = softmax(pred_s, axis=1)
softmax_t = softmax(pred_t, axis=1)

labels = data_provider.train_labels(20)

model = trainer.model

low_s = model.encoder(torch.from_numpy(train_data_s).to(device=DEVICE).float()).detach().cpu().numpy()
low_t = model.encoder(torch.from_numpy(train_data_t).to(device=DEVICE).float()).detach().cpu().numpy()

In [ ]:
# normalize low_t
y_max = max(low_s[:, 1].max(), low_t[:, 1].max())
y_min = max(low_s[:, 1].min(), low_t[:, 1].min())
x_max = max(low_s[:, 0].max(), low_t[:, 0].max())
x_min = max(low_s[:, 0].min(), low_t[:, 0].min())
scale = min(100/(x_max - x_min), 100/(y_max - y_min))
low_t = low_t*scale
low_s = low_s*scale

In [ ]:
high_dists = np.linalg.norm(train_data_s-train_data_t, axis=1)
softmax_dists = np.array([js_div(softmax_s[i], softmax_t[i]) for i in range(len(softmax_t))])
euclidean_dists = np.linalg.norm(low_s-low_t, axis=1)
# cosine_dists = np.array([cosine(low_t[i], low_s[i]) for i in range(len(low_s))])

In [ ]:
# find the minimum distance
from pynndescent import NNDescent
# number of trees in random projection forest
n_trees = min(64, 5 + int(round((train_data_t.shape[0]) ** 0.5 / 20.0)))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data_t.shape[0]))))
# distance metric
metric = "euclidean"
# metric = "cosine"
# get nearest neighbors

nnd = NNDescent(
    train_data_t,
    n_neighbors=2,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=False
)
knn_indices, knn_dists = nnd.neighbor_graph

In [ ]:
nn_ind = knn_dists[:, 1]
nn_dists = knn_dists[:, 1]

In [ ]:
threshold = nn_dists.max()
threshold = 0.5
if np.sum(selected) == 0:
    print("No fixing points!")
else:
    print("euclidean dists")
    print(np.sum(euclidean_dists[selected]<low_threshold), np.sum(selected))
    print(euclidean_dists[selected].min(), euclidean_dists.min())
    print(euclidean_dists[selected].mean(), euclidean_dists.mean())
    print(euclidean_dists[selected].max(), euclidean_dists.max())

In [ ]:
softmax_diff = np.array([js_div(softmax_s[i], softmax_s[j]) for (i,j) in knn_indices])
threshold = softmax_diff.max()
selected = softmax_dists<=threshold
if np.sum(selected) == 0:
    print("No fixing points!")
else:
# selected = np.argsort(high_dists)[:100]
    print("euclidean dists")
    print(euclidean_dists[selected].min(), euclidean_dists.min())
    print(euclidean_dists[selected].mean(), euclidean_dists.mean())
    print(euclidean_dists[selected].max(), euclidean_dists.max())

# Graph mutual information
**node and features**
- topology
- mutual information
>>
intrinsic dimension
softmax kl div